# Project Empower

### The aim of this project is to combat unemployment rate in Nigeria through suggestion of reasonable means of utilization of individual skills to fully eliminate dependency on scarce jobs. We expose job seekers to experts in a desired field for better productivity. 

We will attempt to fetch information from a database(In our case pgAdmin 4 and postgresql as sql) using the psycopg2 library. We will use TraitsUI library to display the result.

In [1]:
import psycopg2 as pg2

In [2]:
from traits.api import HasTraits, Str, Range, Enum, Int, Button
from traitsui.api import Item, RangeEditor, View, Label

In [3]:
my_password = 'Your database password goes here'

In [4]:
#connecting to the database
conn = pg2.connect(database="Empower", user="postgres", password=my_password)

In [5]:
cur = conn.cursor()

Let's get the first five content from our database

In [6]:
#we have a table named Business in our database
cur.execute("Select * from Business Limit 5;")
data = cur.fetchall()

In [9]:
for dt in data:
    print(dt)

(1, 1, 'Vulcanizing/Tyre repairs and maintenance', 'Vulcanizing/Tyre repairs and maintenance', 'technical know how', '50k', '50k', '80k monthly/ 960k yearly', 3, 'nan', '9030473437', 'nan', 'Get a larger space', 50000, 50000)
(2, 6, 'Uzu Driving School Nsukka', 'Driving School', 'Driving Skills and Communication skills', '150k', '30k', '80k monthly/ 960k yearly', 2, 'nan', '8037755083', '500k', 'Get more cars', 150000, 30000)
(3, 6, 'Laptop Importation/Sales', 'Laptop Importation/Sales', 'Marketing Skills', '300k', '30k', '80k monthly/ 960k yearly', 2, 'nan', '90242091406', '1 million', 'get more branches', 300000, 30000)
(4, 6, 'Dj palata Djing services', 'Djing Services', 'technical know how', '200k', '20k', '80k monthly/ 960k yearly', 0, 'nan', '8064290423', '400k', 'Buy advanced equipments', 200000, 20000)
(5, 4, 'Mandela Private Fitness Trainer', 'Private Fitness Trainer', 'technical know how and Patience', '40k', '20k', '80k monthly/ 960k yearly', 1, 'nan', '8039520797', '300k', 

To have a proper view of our data, we will use the pandas library to display it from a csv file.

In [10]:
import numpy as np
import pandas as pd

In [12]:
empow = pd.read_csv('businessData.csv')
empow.head()

,business id,business name,business type,important skill,fund required,monthly expenditure,monthly annual profit,num staff,major product,contact phone,expansion capital,firstexpansion strategy
0,1,Vulcanizing/Tyre repairs and maintenance,Vulcanizing/Tyre repairs and maintenance,technical know how,50k,50k,80k monthly/ 960k yearly,3,NaN,9030473437,NaN,Get a larger space
1,2,Uzu Driving School Nsukka,Driving School,Driving Skills and Communication skills,150k,30k,80k monthly/ 960k yearly,2,NaN,8037755083,500k,Get more cars
2,3,Laptop Importation/Sales,Laptop Importation/Sales,Marketing Skills,300k,30k,80k monthly/ 960k yearly,2,NaN,90242091406,1 million,get more branches
3,4,Dj palata Djing services,Djing Services,technical know how,200k,20k,80k monthly/ 960k yearly,0,NaN,8064290423,400k,Buy advanced equipments
4,5,Mandela Private Fitness Trainer,Private Fitness Trainer,technical know how and Patience,40k,20k,80k monthly/ 960k yearly,1,NaN,8039520797,300k,get a big hall


We will now attempt to fetch information from a database based on 3 criterias. Number of staffs required, fund required to start the business and the monthly expenditure. 

Let's categories the above criteria and see how it looks.

In [19]:
class Empower(HasTraits):
    number_of_staffs = Enum('1 - 10','11 - 20','21 - 50')
    fund_required = Enum('1k - 50k','51k - 200k','210k - 1M','1.1M - 50M','51M - 100M')
    monthly_expenditure = Enum('1k - 50k','51k - 200k','210k - 1M','1.1M - 15M')
    
empower = Empower()

Empower_view = View(
    Label('Select the options that best suits you.'),
    Item('number_of_staffs', width = 150),
    Item('fund_required', width = 150),
    Item('monthly_expenditure', width = 150),
    buttons=['OK', 'Cancel'],
    title = 'Combating Unemployment through Youth Empowerment',
    resizable = True
)
empower.configure_traits(view=Empower_view)

True

Let's create a simple function to extract details from our VIEW 

In [14]:
def getLimit(param):
    x,y = param.split(' - ')
    if 'k' in x:
        x = float(x[:-1]) * 1000
    elif 'M' in x:
        x = float(x[:-1]) * 1000000
    if 'k' in y:
        y = float(y[:-1]) * 1000
    elif 'M' in y:
        y = float(y[:-1]) * 1000000
    return [int(x),int(y)]
getLimit('210k - 1M')

[210000, 1000000]

In [20]:
getLimit(empower.fund_required)

[1000, 50000]

Now we will create another function to extract information from the database using information from our VIEW.

In [16]:
def fetchData(number_of_staffs,fund_required,monthly_expenditure):
    k1 = getLimit(empower.number_of_staffs)
    k2 = getLimit(empower.fund_required)
    k3 = getLimit(empower.monthly_expenditure)

    q1 = 'select business_name,important_skill,fund_required,monthly_annual_profit,contact_phone from Business '
    q2 = 'where number_staff between ' + str(k1[0]) + ' and ' + str(k2[1]) + ' and '
    q3 = 'actual_fund between ' + str(k2[0]) + ' and ' + str(k2[1]) + ' and '
    q4 = 'actual_monthly_expenditure between ' + str(k3[0]) + ' and ' + str(k3[1]) + ';'
    qry = q1 + q2 + q3 + q4

    cur.execute(qry)
    details = cur.fetchall()
    return details

We will create another function to format our generated result.

In [17]:
def Result(det):
    res = 'Business Name| Important Skill| Fund Required| Monthly Annual Profit| Expert Phone Number \n'
    for dt in det:
        res += '| '.join(dt)
        res += '\n'
    return res

Let us get information from the database based on our above selection

In [21]:
details = fetchData(empower.number_of_staffs,empower.fund_required,empower.monthly_expenditure)
print(Result(details))

Business Name| Important Skill| Fund Required| Monthly Annual Profit| Expert Phone Number 
Vulcanizing/Tyre repairs and maintenance| technical know how| 50k| 80k monthly/ 960k yearly| 9030473437
Mandela Private Fitness Trainer| technical know how and Patience| 40k| 80k monthly/ 960k yearly| 8039520797
Korede Butcher/ Meat Dealer| technical know how| 30k| 80k monthly/ 960k yearly| 8164610970
Idoko Motorcycle Repairs| technical know how| 50k| 60k/720k| 8031337992
Seyi Phone Repairs| technical know how| 40k| 60k/720k| 8184800116
Fruit Emporium Fruit sales| Managerial Skills| 50k| 60k/720k| 8034471367
Fruit Emporium Fruit sales| Managerial Skills| 50k| 60k/720k| 8034471367
Fruit Emporium Fruit sales| Managerial Skills| 50k| 60k/720k| 8034471367
Forever bee honey centre| technical know how| 50k| 50k/600k| 8146467751
Mayreez Crafts Business| technical know how| 40k| 50k/600k| 7065837854
Mama Malik Charcoal and Firewood| Managerial Skills and technical know how| 40k| 50k/600k| 8098073574
Ara 

Let's complete the the GUI to fetch data from the database and display the result

In [23]:
class Empower(HasTraits):
    number_of_staffs = Enum('1 - 10','11 - 20','21 - 50')
    fund_required = Enum('1k - 50k','51k - 200k','210k - 1M','1.1M - 50M','51M - 100M')
    monthly_expenditure = Enum('1k - 50k','51k - 200k','210k - 1M','1.1M - 15M')
    
empower = Empower()

Empower_view = View(
    Label('Select the options that best suits you.'),
    Item('number_of_staffs', width = 150),
    Item('fund_required', width = 150),
    Item('monthly_expenditure', width = 150),
    buttons=['OK', 'Cancel'],
    title = 'Combating Unemployment through Youth Empowerment',
    resizable = True
)

class AllDetails(HasTraits):
    name = Str
    
alldetails = AllDetails()

if empower.configure_traits(view=Empower_view):
    ftd = fetchData(empower.number_of_staffs,empower.fund_required,empower.monthly_expenditure)
    det = Result(ftd)
    AllDetails_view = View(
        Label('Details of your Selection'),
        Label(det),
        buttons=['OK', 'Cancel'],
        title = 'Entrepreneur Details',
        resizable = True
    )
    alldetails.configure_traits(view=AllDetails_view)

We will attempt to search a business type rather than using the above criteria.
We will create a function to search the database.

In [24]:
def fetchSearch(s):
    qr = 'select business_name,important_skill,fund_required,monthly_annual_profit,contact_phone from Business '
    qr1 = "where business_type ILIKE '%" +s+ "%';"
    qry = qr + qr1
    cur.execute(qry)
    details = cur.fetchall()
    return details

In [28]:
class SearchDetails(HasTraits):
    name = Str
    
searchdetails = SearchDetails()

class Search(HasTraits):
    name = Str
    
search = Search()

Search_view = View(
    Label('What business do you have in mind?'),
    Item('name', width = 150),
    buttons=['OK', 'Cancel'],
    title = 'Combating Unemployment through Youth Empowerment',
    resizable = True
)


if search.configure_traits(view=Search_view):
    ftd = fetchSearch(search.name)
    det = Result(ftd)
    SearchDetails_view = View(
        Label('Details of your Selection'),
        Label(det),
        buttons=['OK', 'Cancel'],
        title = 'Entrepreneur Details',
        resizable = True
    )
    searchdetails.configure_traits(view=SearchDetails_view)

We will attempt to implement the two criterias on a single View

In [30]:
class Empowerment(HasTraits):
    use_search_button = Button('Click here')
    number_of_staffs = Enum('1 - 10','11 - 20','21 - 50')
    fund_required = Enum('1k - 50k','51k - 200k','210k - 1M','1.1M - 50M','51M - 100M')
    monthly_expenditure = Enum('1k - 50k','51k - 200k','210k - 1M','1.1M - 15M')
    
    def _use_search_button_fired(self):
        if search.configure_traits(view=Search_view):
            ftd = fetchSearch(search.name)
            det = Result(ftd)
            SearchDetails_view = View(
                Label('Details of your Selection'),
                Label(det),
                buttons=['OK', 'Cancel'],
                title = 'Entrepreneur Details',
                resizable = True
            )
            searchdetails.configure_traits(view=SearchDetails_view)

    traits_view = View(
    'use_search_button',
    Label('Select the options that best suits you.'),
    Item('number_of_staffs', width = 150),
    Item('fund_required', width = 150),
    Item('monthly_expenditure', width = 150),
    buttons=['OK', 'Cancel'],
    title = 'Combating Unemployment through Youth Empowerment',
    resizable = True
    )
empowerment = Empowerment()
if empowerment.configure_traits():
    ftd = fetchData(empowerment.number_of_staffs,empowerment.fund_required,empowerment.monthly_expenditure)
    det = Result(ftd)
    AllDetails_view = View(
        Label('Details of your Selection'),
        Label(det),
        buttons=['OK', 'Cancel'],
        title = 'Entrepreneur Details',
        resizable = True
    )
    alldetails.configure_traits(view=AllDetails_view)

# Thank You